In [1]:
# region Imports
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
# endregion

In [2]:
# region Data Ingestion
# Retrieve USGS Dataset
usgs_df = pd.read_csv("../data/all_month.csv")

# endregion
usgs_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2024-06-03T14:46:11.670Z,38.806000,-122.821663,3.56,0.75,md,7.0,138.0,0.006533,0.01,...,2024-06-03T14:47:50.654Z,"6 km NW of The Geysers, CA",earthquake,0.70,1.84,0.120000,9.0,automatic,nc,nc
1,2024-06-03T14:43:15.230Z,19.362000,-155.222672,3.13,1.74,md,7.0,255.0,0.013540,0.15,...,2024-06-03T14:44:59.200Z,"9 km S of Volcano, Hawaii",earthquake,1.09,0.40,0.170000,6.0,automatic,hv,hv
2,2024-06-03T14:34:20.449Z,61.953100,-149.505500,18.20,1.30,ml,NaN,NaN,NaN,0.73,...,2024-06-03T14:37:13.541Z,"27 km NNW of Fishhook, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
3,2024-06-03T14:30:57.060Z,19.370667,-155.227493,0.82,2.03,md,21.0,97.0,0.011740,0.16,...,2024-06-03T14:32:24.870Z,"8 km S of Volcano, Hawaii",earthquake,0.27,0.19,0.340000,13.0,automatic,hv,hv
4,2024-06-03T14:24:55.470Z,19.168167,-155.481995,31.15,2.25,md,35.0,95.0,0.022680,0.10,...,2024-06-03T14:27:18.020Z,"3 km S of Pāhala, Hawaii",earthquake,0.57,0.73,0.040000,3.0,automatic,hv,hv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9863,2024-05-04T15:04:20.323Z,59.540600,-151.977300,58.90,1.80,ml,NaN,NaN,NaN,0.36,...,2024-05-04T15:06:33.541Z,"18 km NW of Seldovia, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak
9864,2024-05-04T15:00:14.740Z,33.583333,-116.807667,7.53,0.18,ml,10.0,87.0,0.037810,0.09,...,2024-05-07T01:13:53.032Z,"13 km WNW of Anza, CA",earthquake,0.30,0.67,0.051000,4.0,reviewed,ci,ci
9865,2024-05-04T14:58:47.080Z,33.583500,-116.805667,7.69,0.48,ml,15.0,73.0,0.036160,0.13,...,2024-05-07T01:04:49.305Z,"13 km WNW of Anza, CA",earthquake,0.28,0.81,0.082000,5.0,reviewed,ci,ci
9866,2024-05-04T14:49:58.580Z,32.775167,-115.443833,11.06,1.97,ml,37.0,53.0,0.030170,0.29,...,2024-05-07T13:23:42.579Z,"7 km SW of Holtville, CA",earthquake,0.34,0.48,0.285000,35.0,reviewed,ci,ci


In [27]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 33.6 MB/s eta 0:00:00


In [49]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load your dataset
df = pd.read_csv("../data/all_month.csv")
df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
df['mag_size'] = df['mag'].clip(0.1)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='map-plot', config={'scrollZoom': True}),
    dcc.Graph(id='time-series-plot'),
    dcc.Graph(id='magnitude-distribution-plot')
])

@app.callback(
    [
        Output('map-plot', 'figure'),
        Output('time-series-plot', 'figure')
    ],
    [
        Input('magnitude-distribution-plot', 'selectedData'),
        Input('time-series-plot', 'relayoutData')
    ]
)
def update_map_and_time_series(selectedMagData, selectedTimeData):
    filtered_df = df

    # Handle time range selection from the time-series plot
    if selectedTimeData and 'xaxis.range[0]' in selectedTimeData:
        start_date = pd.to_datetime(selectedTimeData['xaxis.range[0]']).date()
        end_date = pd.to_datetime(selectedTimeData['xaxis.range[1]']).date()
        filtered_df = filtered_df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    # Handle magnitude selection from the magnitude distribution plot
    if selectedMagData:
        selected_mags = [point['x'] for point in selectedMagData['points']]
        filtered_df = filtered_df[df['mag'].isin(selected_mags)]

    # Update map
    map_fig = px.scatter_geo(
        filtered_df,
        height=900,
        lat='latitude',
        lon='longitude',
        color='mag',
        hover_name='place',
        size='mag_size',
        projection='natural earth',
        title='Map of Earthquakes'
    )

    # Update time series plot
    time_series_fig = px.line(
        filtered_df.groupby('date').size().reset_index(name='count'),
        x='date',
        y='count',
        title='Number of Earthquakes Over Time'
    )

    return map_fig, time_series_fig

@app.callback(
    Output('magnitude-distribution-plot', 'figure'),
    [Input('map-plot', 'selectedData')]
)
def update_magnitude_distribution(selectedData):
    if selectedData:
        points = selectedData['points']
        lats = [point['lat'] for point in points]
        lons = [point['lon'] for point in points]
        filtered_df = df[df['latitude'].isin(lats) & df['longitude'].isin(lons)]
    else:
        filtered_df = df

    mag_dist_fig = px.histogram(
        filtered_df,
        x='mag',
        nbins=40,
        title='Distribution of Earthquake Magnitudes'
    )

    return mag_dist_fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [17]:
df.groupby('date').size().rename("mag")

date
2024-05-04    139
2024-05-05    364
2024-05-06    419
2024-05-07    380
2024-05-08    299
2024-05-09    355
2024-05-10    303
2024-05-11    264
2024-05-12    346
2024-05-13    382
2024-05-14    351
2024-05-15    323
2024-05-16    288
2024-05-17    346
2024-05-18    547
2024-05-19    448
2024-05-20    317
2024-05-21    304
2024-05-22    251
2024-05-23    257
2024-05-24    334
2024-05-25    249
2024-05-26    289
2024-05-27    304
2024-05-28    309
2024-05-29    233
2024-05-30    258
2024-05-31    256
2024-06-01    263
2024-06-02    314
2024-06-03    376
Name: s, dtype: int64